#Exploring the Qiskit SDK for quantum programming through the Greenberger–Horne–Zeilinger Entangeled Quantum State

Qiskit is a software development kit (SDK) providing acomprehensive platform for quantum programming. Qiskit has been developed by IBM.
At the heart of Qiskit are its elements - Terra, Aer, Ignis, and Aqua - each playing a crucial role in the quantum computing ecosystem. In this notebook we work through a baseline quantum simulation using Qiskit, exploring the GHZ entangled quantum state.

In [2]:
# ! pip install qiskit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 13.6 MB/s eta 0:00:00


In [4]:
# https://pypi.org/project/qiskit/

### Imports

In [7]:
import numpy as np
import pandas

import qiskit
from qiskit import QuantumCircuit
from qiskit.primitives.sampler import Sampler
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import Estimator
from qiskit import transpile

### Create the GHZ Entangled State

The entangled state named the GHZ state uses the Hadamard gate (h), the Phase gate (p), and the CNOT gate (cx).

In [8]:
# Add the classical output, a measurement of all qubits
qc_measured = qc_example.measure_all(inplace=False)

# Execute using the Sampler primitive
from qiskit.primitives.sampler import Sampler
sampler = Sampler()
job = sampler.run(qc_measured, shots=1000)
result = job.result()
print(f" > Quasi probability distribution: {result.quasi_dists}")

 > Quasi probability distribution: [{0: 0.515, 7: 0.485}]


In [10]:
# Quantum circuit for preparing the quantum state:
qc_example = QuantumCircuit(3)
qc_example.h(0)          # generate superpostion
qc_example.p(np.pi/2,0)  # add quantum phase
qc_example.cx(0,1)       # 0th-qubit-Controlled-NOT gate on 1st qubit
qc_example.cx(0,2)       # 0th-qubit-Controlled-NOT gate on 2nd qubit

The Greenberger-Horne-Zeilinger (GHZ) state demonstrates some peculiarities of quantum entanglement across
multiple particles or qubits. Initially studied in the context of four particles by Daniel Greenberger, Michael Horne, and Anton Zeilinger in 1989,
and later expanded to three particles by N. David Mermin in 1990, GHZ states involve at least three subsystems that are
interconnected in such a way that the state of each particle cannot be described independently of the others.

In [11]:
# Add the classical output, a measurement of all qubits
qc_measured = qc_example.measure_all(inplace=False)

In [12]:
# Use the Sampler primitive for execution
sampler = Sampler()
job = sampler.run(qc_measured, shots=1000)
result = job.result()
print(f" > Quasi Probability Distribution: {result.quasi_dists}")

 > Quasi Probability Distribution: [{0: 0.523, 7: 0.477}]


In [13]:
# Define the observable to be measured
operator = SparsePauliOp.from_list([("XXY", 1), ("XYX", 1), ("YXX", 1), ("YYY", -1)])

In [14]:
# Use the Estimator primitive for execution
estimator = Estimator()
job = estimator.run(qc_example, operator, shots=1000)
result = job.result()
print(f" > Expectation values: {result.values}")

 > Expectation values: [4.]



Leveraging the Qiskit Sampler and Qiskit Estimator primitives provided by Qiskit has its limitations. The true potential of quantum computing surpasses what can be emulated by classical computers, necessitating the use of actual quantum hardware for executing larger quantum circuits. To run a quantum circuit on real quantum hardware, it's essential to adapt the circuit to the specific basis gates and the connectivity layout of the quantum device. This adaptation is facilitated by a tool known as the transpiler, which Qiskit furnishes with a variety of passes for synthesis, optimization, mapping, and scheduling. Additionally, Qiskit offers a default compiler that efficiently handles most cases, ensuring quantum circuits are suitably prepared for execution on quantum hardware.

In [15]:
qc_transpiled = transpile(qc_example, basis_gates = ['cz', 'sx', 'rz'], coupling_map =[[0, 1], [1, 2]] , optimization_level=3)

### Next step: Experiment with execution on quantum hardware using Qiskit